# Bayesian Network for Melanoma

In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics
import matplotlib.pyplot as plt
from __future__ import division
from scipy import linalg
import itertools
from pyitlib import discrete_random_variable as drv
reload(drv)
import scipy.stats 
import pprint as pp
from pgmpy.factors.discrete import JointProbabilityDistribution as JPD
from pgmpy.models import BayesianModel
from pgmpy.estimators import ConstraintBasedEstimator
from sklearn import metrics
from pgmpy.factors.discrete import TabularCPD, DiscreteFactor
from pgmpy.inference import VariableElimination

The function calculates the conditional mutual infomation between X and Y given some evidence Z :

In [2]:
def conditional_independence(test,X,Y,Z):
    
    X = test[X].values
    Y = test[Y].values
    Z = test[Z].values
    Z= Z.T
    conditional_info = drv.entropy_joint(np.row_stack((X,Z))) + drv.entropy_joint(np.row_stack((Y,Z))) \
                        - drv.entropy_joint(np.row_stack((X,Y,Z))) - drv.entropy_joint(Z)
    return conditional_info

The function tests whether X and Y are independent given Z, it used the chi-square statistic as a measure :

In [3]:
#def conditional_independence(test,X,Y,Z):
    
 #   c = ConstraintBasedEstimator(test)
    
 #   chi2, p_value, sufficient_data = c.test_conditional_independence(X,Y,Z)
    
 #   return p_value

The function defines a witness (single and pairs other than the node under investigation) and checks for conditional independence:

In [4]:
def pdag_test(test,threshold):
    skeleton = {}
    witness = {}
    U = []
    
    labels = test.columns
    pairs = list(itertools.combinations(labels, 2))
    
    for X,Y in pairs:
        witness[X,Y] = [] 
        witness[Y,X] =  []
        skeleton[X,Y] = 1
        skeleton[Y,X] = 1

    for X,Y in pairs:
        tag = []
        wit = []
        singles = []
        for Z in labels:
            if Z!= X and Z!=Y:
                singles.append(Z)
                wit.append(Z)
        
        twos = list(itertools.combinations(singles, 2))
        
        for element in twos:
            wit.append(list(element))
        
        for w in wit:
            U = conditional_independence(test,X,Y,w)
            if U <= threshold: # conditionally independent
                skeleton[X,Y] = 0
                skeleton[Y,X] = 0
                witness[X,Y] = w 
                witness[Y,X] = w
                break 
    
    return skeleton, witness

The function checks for possible immoralities:

In [5]:
def immoralities(test,skeleton,witness):
    
    labels = test.columns
    triples = list(itertools.permutations(labels, 3))

    immorals = []
    for t in triples:
        if (skeleton[(t[0], t[1])] !=0 and skeleton[(t[1], t[2])] !=0 and skeleton[(t[0], t[2])] ==0):
            if t[1] not in witness[(t[0], t[2])]:
                immorals.append((t[0],t[1],t[2]))
                skeleton[(t[0], t[1])] = 10
                skeleton[(t[1], t[0])] = -10
                skeleton[(t[2], t[1])] = 10
                skeleton[(t[1], t[2])] = -10
   
    return immorals,skeleton

This function performs rule propogation to create a PDAG and to ensure that the graph is acyclic:

In [6]:
def cycles(test,skeleton):
    labels = test.columns
    triples = list(itertools.permutations(labels, 3))
    loops = list(itertools.permutations(labels,4))
    while(1):
        check = False
        for t in triples:
            if(skeleton[(t[0], t[1])] ==10 and skeleton[(t[1], t[2])] ==1 and skeleton[(t[0], t[2])] ==0):
                skeleton[(t[1], t[2])] = 10
                skeleton[(t[2], t[1])] = -10
                check = True
            if(skeleton[(t[0], t[1])] ==10 and skeleton[(t[1], t[2])] ==10 and skeleton[(t[0], t[2])] ==1):
                skeleton[(t[0], t[2])] = 10
                skeleton[(t[2], t[0])] = -10
                check = True
        for l in loops:
            if(skeleton[(l[0], l[1])] ==1 and skeleton[(l[0], l[2])] ==1 and skeleton[(l[0], l[3])] ==1 and skeleton[(l[1], l[3])] ==10 and skeleton[(l[2], l[3])] ==10):
                skeleton[(l[0], l[3])] = 10
                skeleton[(l[3], l[0])] = -10
                check = True
        if check == False:
            break
    return skeleton

The function is only written to achieve a presentable print of the structure:

In [7]:
def concise(test,skeleton):

    concise_skeleton = {}

    labels = test.columns
    pairs = list(itertools.combinations(labels, 2))

    for (X,Y) in pairs:
        concise_skeleton[X,Y] = skeleton[X,Y]
        
    return concise_skeleton

The following function gives us a dictionary of each node and its corresponding parents:

In [8]:
def parents(test,skeleton):
    
    parents = {}
    
    labels = test.columns
    pairs = list(itertools.combinations(labels, 2))
    
    parent = []
    
    for X in labels:
        parent = []
        for Y in labels:
            if X != Y:
                if skeleton[X,Y] == -10:
                    parent.append(Y)
            parents[X] = parent
            
    return parents

This function calculates the joint distribution of two variables X and Y:

In [9]:
def joint_distribution(test,X,Y):
    
    X = np.array(test[X].values)
    Y = np.array(test[Y].values)

    B = []
    
    Z = np.column_stack((X,Y))
    
    n = Z.shape[1]
    
    # Since the data is binary, we can assume that it takes a limited number of permutations 
    # The function counts how many instances belong to a particular arrangement amonst the possibilities

    for i in xrange(2**n):
        b = bin(i)[2:]
        l = len(b)
        b = str(0) * (n - l) + b
        B.append(b)
    
    counts = np.zeros(len(B))
    
    l=[int(i) for  element in B for i in element]
    B = [l[i:i+n] for i in xrange(0, len(l), n)]
    
    for j in range(0,len(B)): 
        count = 0
        for k in range(0,len(Z)):
            check = Z[k] == B[j]
            if check.all():
                count = count + 1
        counts[j] = count
    
    probs = counts/Z.shape[0]
    
    return probs,counts,B

This function calculates the marginal distribution of one variable X:

In [10]:
def marginal_dist(test,X):
    
    marginal,counts,possibilities = joint_distribution(test,X,[])
    
    return marginal,counts,possibilities

# Maximum Likelihood Estimation

This function calculates the conditional distribution of X given Y using the MLE approach:

In [11]:
def conditional_dist(test,X,Y):
   
    conditional = []

    joint,jcount,possibilities = joint_distribution(test,X,Y)
    marginal,mcount,possibility = marginal_dist(test,Y)

    num = len(possibility)
    ind = int(len(jcount)/num)
    powers = []
    
    u = []
    
    for i in range(0,num):
        u.append((jcount[i],jcount[i+num]))
    
    theta_x_u = []
    
    # In the MLE approach, a count of 0 will lead to an improper fraction and thus I use Laplacian smoothing
    
    V = test.values.shape[0]
    smooth = 0.0001
    
    for i in range(0,len(u)):
        th = u[i]/(mcount[i]+V*smooth) + smooth/(mcount[i]+V*smooth)
        theta_x_u.append(th)
    
    x_u = [i for  element in theta_x_u for i in element]

    M_x_u = [int(i) for  element in u for i in element]
    
    # Calculate likelihood as the product of the theta that maximises raised to the respective counts 
    
    theta = []
    for i in range(0,len(theta_x_u)):
        theta.append(x_u[i]**M_x_u[i])
    conditional = np.prod(theta)
    
    logtheta = []
    for i in range(0,len(theta_x_u)):
        logtheta.append(np.log(x_u[i])*M_x_u[i])
    logcond = np.sum(logtheta)
    
    cpd = [np.transpose(x_u[i:i+2]) for i in xrange(0, len(x_u),2)]
    cpd = np.transpose(cpd)
    
    return conditional,theta,logcond,cpd

This function just collects the necessary information to define a Bayesian network:

In [12]:
def likelihood_values(test,pdag):
    vals = []
    likelihood = []
    loglike = []
    for X in test.columns :
        parent_set = parents(test,pdag)
        Y = parent_set[X]
        if not Y:
            probs,count,b = joint_distribution(test,X,[])
            vals.append((X,[],probs))
            likelihood.append(np.prod(probs))
            loglike.append(np.sum(np.log(probs)))
            continue
        else:
            cond,th,logcond,cpd = conditional_dist(test,X,Y)
            vals.append((X,Y,cpd))
            likelihood.append(cond)
            loglike.append(logcond)
    
    return vals,likelihood,loglike

# Bayesian Parameter Estimation

The function gives the values of alpha for the Dirichlet priors for each node given its parents:

In [13]:
def alphas(test,pdag,d):
    d = 1

    alph = {}
    for X in test.columns :
            parent_set = parents(test,pdag)
            Y = parent_set[X]
            if not Y:
                alph[X] = [d,d]
                continue
            else:
                labels = np.append([X],[Y])
                card = d*np.ones(2**len(Y))
                alph[X] = np.row_stack((card,card))
    
    return alph

The function calculates the posterior distribution, which due to conjugacy is also a Dirichlet distribution : 

In [14]:
def bayesian_parameters(test,X,Y,alpha):
    
    A = alpha[X]
    
    joint,jcount,possibilities = joint_distribution(test,X,Y)
    
    if not Y:
        cpd = jcount
        post_num = A+cpd
        post = post_num/np.sum(post_num)
    else:
        n = int(len(jcount)/2)
        cpd = [jcount[i:i+n] for i in xrange(0, len(jcount),n)]
    
        cpd = np.row_stack(cpd)
    
        post_num = A+cpd
        P = np.zeros((post_num.shape[1],post_num.shape[0]))

        for i in range(0,post_num.shape[1]):
            p = np.transpose(post_num)
            P[i] = p[i]/np.sum(p[i])
    
        post = np.transpose(P)
    
    
    return post

This function accumulates all the information required to define a Bayesian network:

In [15]:
def bayesian_likelihood(test,pdag,alpha):
    vals = []
    loglike = []
    dim = []
    for X in test.columns :
        parent_set = parents(test,pdag)
        Y = parent_set[X]
        if not Y:
            cpd = bayesian_parameters(test,X,[],alpha)
            vals.append((X,[],cpd))
            loglike.append(np.sum(np.log(cpd)))
            dim.append(X)
            continue
        else:
            cpd = bayesian_parameters(test,X,Y,alpha)
            vals.append((X,Y,cpd))
            loglike.append(np.sum(np.log(cpd)))
    
    return vals,loglike,dim

# BIC Score

In [16]:
def bic_score(test,like,dim):
    
    score = np.prod(like) - np.log(test.columns.shape[0])*len(dim)/2
    
    return score

# Defining the Bayesian network

In [17]:
def define_network(test,pdag,vals):
    G = BayesianModel()

    G.add_nodes_from(test.columns)

    pairs = list(itertools.combinations(test.columns, 2))

    cpd_add = []
    for X in test.columns :
        parent_set = parents(test,pdag)
        Y = parent_set[X]
        if not Y:
            cpd_add.append(TabularCPD(variable=X, variable_card=2,
                      values=[vals[int(X)][2]]))
            continue
        else:
            G.add_edges_from([(y,X) for y in Y])
            card = len(Y)
            e_card = [2 for i in range(0,len(Y))]
            cpd_add.append(TabularCPD(variable=X, variable_card=2, 
                      values=vals[int(X)][2],evidence=vals[int(X)][1],
                        evidence_card=e_card))

    for t in cpd_add:
        G.add_cpds(t)
        
    if G.check_model():
        return G

Load the data from the text file into a dataframe object:

In [18]:
R = np.loadtxt('expressions.txt')
test = pd.DataFrame(data=np.transpose(R))
cols = [str(x) for x in test.columns.tolist()]
test = pd.DataFrame(data=np.transpose(R),columns = cols)

Find the structure :

In [20]:
skeleton = np.load('skeleton_final.npy').item()
witness = np.load('witness_final.npy').item()

In [21]:
#threshold = 0.02
#skeleton, witness = pdag_test(test,threshold)
immorals, skeleton_new = immoralities(test,skeleton,witness)
pdag = cycles(test,skeleton_new)

Print the final structure:

In [22]:
concise_skeleton = concise(test,pdag)
pp.pprint(concise_skeleton)

{('0', '1'): 0,
 ('0', '10'): 0,
 ('0', '11'): 0,
 ('0', '12'): 0,
 ('0', '13'): 0,
 ('0', '14'): 0,
 ('0', '15'): 0,
 ('0', '16'): 0,
 ('0', '17'): 0,
 ('0', '18'): 0,
 ('0', '19'): 0,
 ('0', '2'): 0,
 ('0', '20'): 0,
 ('0', '21'): 0,
 ('0', '22'): 0,
 ('0', '23'): 0,
 ('0', '24'): 0,
 ('0', '25'): 0,
 ('0', '26'): 0,
 ('0', '27'): 0,
 ('0', '28'): 0,
 ('0', '29'): 0,
 ('0', '3'): 0,
 ('0', '30'): 0,
 ('0', '31'): 0,
 ('0', '32'): 0,
 ('0', '33'): 0,
 ('0', '34'): 0,
 ('0', '35'): 0,
 ('0', '36'): 0,
 ('0', '37'): 0,
 ('0', '38'): 0,
 ('0', '39'): 0,
 ('0', '4'): 0,
 ('0', '40'): 0,
 ('0', '41'): 0,
 ('0', '42'): 0,
 ('0', '43'): 0,
 ('0', '44'): 0,
 ('0', '45'): 0,
 ('0', '46'): 0,
 ('0', '47'): 0,
 ('0', '48'): 0,
 ('0', '49'): 0,
 ('0', '5'): 0,
 ('0', '50'): 0,
 ('0', '51'): 0,
 ('0', '52'): 0,
 ('0', '53'): 0,
 ('0', '54'): 0,
 ('0', '55'): 0,
 ('0', '56'): 0,
 ('0', '57'): 0,
 ('0', '58'): 0,
 ('0', '59'): 0,
 ('0', '6'): 0,
 ('0', '60'): 0,
 ('0', '61'): 0,
 ('0', '62'): 0,
 ('

 ('21', '22'): 0,
 ('21', '23'): 0,
 ('21', '24'): 0,
 ('21', '25'): 0,
 ('21', '26'): 0,
 ('21', '27'): 0,
 ('21', '28'): 0,
 ('21', '29'): 0,
 ('21', '30'): 0,
 ('21', '31'): 0,
 ('21', '32'): 0,
 ('21', '33'): 0,
 ('21', '34'): 0,
 ('21', '35'): 0,
 ('21', '36'): 0,
 ('21', '37'): 0,
 ('21', '38'): 0,
 ('21', '39'): 0,
 ('21', '40'): 0,
 ('21', '41'): 0,
 ('21', '42'): 0,
 ('21', '43'): 0,
 ('21', '44'): 0,
 ('21', '45'): 0,
 ('21', '46'): 0,
 ('21', '47'): 0,
 ('21', '48'): 0,
 ('21', '49'): 0,
 ('21', '50'): 0,
 ('21', '51'): 0,
 ('21', '52'): 0,
 ('21', '53'): 0,
 ('21', '54'): 0,
 ('21', '55'): 0,
 ('21', '56'): 0,
 ('21', '57'): 0,
 ('21', '58'): 10,
 ('21', '59'): 0,
 ('21', '60'): 0,
 ('21', '61'): 0,
 ('21', '62'): 0,
 ('21', '63'): 0,
 ('21', '64'): 0,
 ('21', '65'): 0,
 ('21', '66'): 0,
 ('21', '67'): -10,
 ('21', '68'): 0,
 ('21', '69'): 0,
 ('21', '70'): 0,
 ('21', '71'): 0,
 ('21', '72'): 0,
 ('21', '73'): 0,
 ('21', '74'): 0,
 ('21', '75'): 0,
 ('21', '76'): 0,
 ('22',

 ('46', '50'): 0,
 ('46', '51'): 0,
 ('46', '52'): 0,
 ('46', '53'): 0,
 ('46', '54'): 0,
 ('46', '55'): 0,
 ('46', '56'): 0,
 ('46', '57'): 0,
 ('46', '58'): 0,
 ('46', '59'): 0,
 ('46', '60'): 0,
 ('46', '61'): 0,
 ('46', '62'): -10,
 ('46', '63'): 0,
 ('46', '64'): 0,
 ('46', '65'): 0,
 ('46', '66'): 0,
 ('46', '67'): 0,
 ('46', '68'): 0,
 ('46', '69'): 0,
 ('46', '70'): 0,
 ('46', '71'): 0,
 ('46', '72'): 0,
 ('46', '73'): 0,
 ('46', '74'): 0,
 ('46', '75'): 0,
 ('46', '76'): 0,
 ('47', '48'): 0,
 ('47', '49'): 0,
 ('47', '50'): 0,
 ('47', '51'): 0,
 ('47', '52'): 0,
 ('47', '53'): 0,
 ('47', '54'): -10,
 ('47', '55'): 0,
 ('47', '56'): 0,
 ('47', '57'): 0,
 ('47', '58'): 0,
 ('47', '59'): 0,
 ('47', '60'): 0,
 ('47', '61'): -10,
 ('47', '62'): 0,
 ('47', '63'): 0,
 ('47', '64'): 0,
 ('47', '65'): 0,
 ('47', '66'): 0,
 ('47', '67'): 0,
 ('47', '68'): 0,
 ('47', '69'): 0,
 ('47', '70'): 0,
 ('47', '71'): 0,
 ('47', '72'): 0,
 ('47', '73'): 0,
 ('47', '74'): 0,
 ('47', '75'): 0,
 ('4

Perform the parameter estimation:

In [43]:
alph = alphas(test,pdag,10)
vals,like,dim= bayesian_likelihood(test,pdag,alph)
print(np.prod(like))

-6.61595826705e+27


Calculate the score:

In [44]:
score = bic_score(test,like,dim)
print(score)

-6.61595826705e+27


# Inference

In [45]:
G = define_network(test,pdag,vals)

In [46]:
inference = VariableElimination(G)

Apoptosis Factors influenced by STAT3

In [47]:
phi_query = inference.query(['2', '3','5','76'],evidence={'70':0})
phi_copy = phi_query.copy()
for X in phi_copy:
    print((X,phi_copy[X].values))

('76', array([ 0.33962264,  0.66037736]))
('3', array([ 0.47169811,  0.52830189]))
('2', array([ 0.45283019,  0.54716981]))
('5', array([ 0.54716981,  0.45283019]))


In [49]:
phi_query = inference.query(['2', '3','5','76'],evidence={'70':1})
phi_copy = phi_query.copy()
for X in phi_copy:
    print((X,phi_copy[X].values))

('76', array([ 0.33962264,  0.66037736]))
('3', array([ 0.47169811,  0.52830189]))
('2', array([ 0.45283019,  0.54716981]))
('5', array([ 0.54716981,  0.45283019]))


Apoptosis Factors influenced by Growth Factors and Death Ligands

In [50]:
phi_query = inference.query(['2', '3','5','76'],evidence={'73':0,'74':0,'21':1,'32':1,'48':1})
phi_copy = phi_query.copy()
for X in phi_copy:
    print((X,phi_copy[X].values))

('76', array([ 0.33962264,  0.66037736]))
('3', array([ 0.47169811,  0.52830189]))
('2', array([ 0.45283019,  0.54716981]))
('5', array([ 0.54716981,  0.45283019]))


Proliferation Factors influenced by Growth Factors and Death Ligands

In [33]:
phi_query = inference.query(['11', '14','15'],evidence={'73':0,'74':0,'21':1,'32':1,'48':1})
phi_copy = phi_query.copy()
for X in phi_copy:
    print((X,phi_copy[X].values))

('11', array([ 0.48199318,  0.51800682]))
('15', array([ 0.30188679,  0.69811321]))
('14', array([ 0.50943396,  0.49056604]))


In [35]:
phi_query = inference.query(['11', '14','15'],evidence={'73':1,'74':1,'21':0,'32':0,'48':0})
phi_copy = phi_query.copy()
for X in phi_copy:
    print((X,phi_copy[X].values))

('11', array([ 0.57012259,  0.42987741]))
('15', array([ 0.30188679,  0.69811321]))
('14', array([ 0.50943396,  0.49056604]))


In [51]:
phi_query = inference.query(['11', '14','15'],evidence={'74':1})
phi_copy = phi_query.copy()
for X in phi_copy:
    print((X,phi_copy[X].values))

('11', array([ 0.52248333,  0.47751667]))
('15', array([ 0.30188679,  0.69811321]))
('14', array([ 0.50943396,  0.49056604]))


Proliferation Factors influenced by Mutant Genes

In [37]:
phi_query = inference.query(['11', '14','15'],evidence={'4':0,'52':0,'60':1,'75':1})
phi_copy = phi_query.copy()
for X in phi_copy:
    print((X,phi_copy[X].values))

('11', array([ 0.52248333,  0.47751667]))
('15', array([ 0.30188679,  0.69811321]))
('14', array([ 0.50943396,  0.49056604]))


Apoptosis Factors influenced by Mutant Genes

In [39]:
phi_query = inference.query(['2', '3','5','76'],evidence={'4':0,'52':0,'60':1,'75':1})
phi_copy = phi_query.copy()
for X in phi_copy:
    print((X,phi_copy[X].values))

('76', array([ 0.33962264,  0.66037736]))
('3', array([ 0.47169811,  0.52830189]))
('2', array([ 0.45283019,  0.54716981]))
('5', array([ 0.54716981,  0.45283019]))
